In [1]:
from fbprophet import Prophet
from fbprophet.diagnostics import cross_validation, performance_metrics
from fbprophet.plot import plot_cross_validation_metric

from sklearn.metrics import mean_absolute_error
import numpy as np


In [2]:
import os
import pandas as pd
from utils import (
    ESTACIONES_XLSX,
    NCEP_NCAR_XLSX,
    NDEFM_XLSX,
    PREDICTIONS_FOLDER
)

2020-03-05 21:07:50,994 - youconfigme.youconfigme - INFO - searching for config on /home/nox/repos/metadata_lluvias/settings.ini


In [3]:
df_est = pd.read_excel(ESTACIONES_XLSX, sheet_name=1, header=1, parse_dates=['Año hid'])
df_est.head()

,Año hid,San Luis,Miraflores,Pozo Sarmiento,Pie de Medano,Valle Viejo,Caimancito,Casa de Piedra,Potrero del Clavillo,Piedra Grande,Los Sosa,Padilla,La Maria,San Luis Tucuman
0,1946-01-01,NaN,NaN,133.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1947-01-01,NaN,NaN,84.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1948-01-01,NaN,NaN,76.3,NaN,NaN,41.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1949-01-01,NaN,NaN,64.4,NaN,NaN,64.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1950-01-01,NaN,NaN,60.0,NaN,NaN,134.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
from datetime import datetime as dt

df_pred = df_est[['Año hid', 'San Luis Tucuman']].rename(columns={'Año hid':'ds', 'San Luis Tucuman':'y'})
df_pred = df_pred[df_pred.ds >= dt(1965, 1, 1)]
df_pred = df_pred.fillna(df_pred.y.mean())

In [5]:
df_train, df_test = df_pred[:-10], df_pred[-10:]

# Asi nomas

In [6]:
m = Prophet(growth='linear',
            yearly_seasonality=True,
            weekly_seasonality=False,
            daily_seasonality=False,
            n_changepoints=25,
            seasonality_mode='multiplicative',
            mcmc_samples=0)
m.fit(df_train)
future = m.make_future_dataframe(periods=10, freq='YS')
fcst = m.predict(future)
mean_absolute_error(df_test.y, fcst[-10:].yhat)

INFO:fbprophet:n_changepoints greater than number of observations. Using 23.


29.794187554580457

In [7]:
fcst[-10:][['yhat']].reset_index(drop=True).to_csv(os.path.join(PREDICTIONS_FOLDER, 'simple_prophecy.csv'), header=False)